In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from features.models import GCNNModelSmall, GCNNModelMedium, GCNNModelLarge
from classes.models import LinearClassifier
from datasets import load_cifar100
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar100(augment=True)

CIFAR-100 loaded in 99 seconds


In [3]:
def test_model(feature_model, settings, data, num_features, num_classes, feature_samplers, classification_samplers,
               f_num_steps, f_lr, f_keep_prob, f_reg, c_num_steps, c_lr, c_reg):
    
    tf.reset_default_graph()
    
    f_train, f_valid, f_test = feature_samplers
    c_train, c_valid, c_test = classification_samplers
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])    
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    f_model = feature_model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=f_lr, reg=f_reg, dropout=dropout, training=training)
    
    features = tf.placeholder(tf.float32, shape=[None, num_features])
    c_model = LinearClassifier(x=features, y=y, num_features=num_features, num_classes=num_features, 
                               lr=c_lr, reg=c_reg)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g, dropout: %g" %(num_features, f_lr, f_reg, f_keep_prob))
        train_time = time.time()
        for step in range(f_num_steps):        
            x_, y_ = f_train.sample(700)
            sess.run(f_model.optimize, feed_dict={x:x_, y:y_, dropout:f_keep_prob, training:True})     
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([f_model.loss, f_model.acc], 
                                                 feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_valid.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        print("begin training classifier")
        train_time = time.time()
        for step in range(c_num_steps):        
            x_, y_ = c_train.sample(700)
            features_ = sess.run(f_model.features, feed_dict={x:x_, dropout:1.0, training:False})
            sess.run(c_model.optimize, feed_dict={features:features_, y:y_, x:x_, dropout:1.0, training:False})
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([c_model.loss, c_model.acc], feed_dict={features:features_, y:y_})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training classifier // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_valid, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_valid})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_test, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_test})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))

In [4]:
test_model(feature_model=GCNNModelSmall, settings=Baseline_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=samplers, classification_samplers=samplers,           
           f_num_steps=50000, f_lr=0.005, f_keep_prob=0.7, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.005, reg: 0.001, dropout: 0.7
	step 0: train loss 5.1756, train error 0.994286
	step 1000: train loss 4.70449, train error 0.928571
	step 2000: train loss 4.48616, train error 0.895714
	step 3000: train loss 4.28408, train error 0.858571
	step 4000: train loss 4.14724, train error 0.86
	step 5000: train loss 4.05787, train error 0.837143
	step 6000: train loss 3.96254, train error 0.794286
	step 7000: train loss 3.81377, train error 0.801429
	step 8000: train loss 3.6956, train error 0.771429
	step 9000: train loss 3.55967, train error 0.74
	step 10000: train loss 3.60746, train error 0.762857
	step 11000: train loss 3.50842, train error 0.751429
	step 12000: train loss 3.33077, train error 0.691429
	step 13000: train loss 3.37124, train error 0.747143
	step 14000: train loss 3.23313, train error 0.725714
	step 15000: train loss 3.18991, train error 0.691429
	step 16000: train loss 3.08696, train error 0.668571
	step 17000: train loss

In [5]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=50000, f_lr=0.005, f_keep_prob=0.7, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.005, reg: 0.001, dropout: 0.7
	step 0: train loss 1.53351, train error 0.505714
	step 1000: train loss 1.46798, train error 0.352857
	step 2000: train loss 1.33485, train error 0.362857
	step 3000: train loss 1.30228, train error 0.372857
	step 4000: train loss 1.24266, train error 0.311429
	step 5000: train loss 1.20417, train error 0.341429
	step 6000: train loss 1.19621, train error 0.324286
	step 7000: train loss 1.17528, train error 0.33
	step 8000: train loss 1.16044, train error 0.345714
	step 9000: train loss 1.10267, train error 0.307143
	step 10000: train loss 1.10224, train error 0.317143
	step 11000: train loss 1.12373, train error 0.324286
	step 12000: train loss 1.0809, train error 0.325714
	step 13000: train loss 1.06041, train error 0.3
	step 14000: train loss 1.08949, train error 0.321429
	step 15000: train loss 1.00825, train error 0.291429
	step 16000: train loss 1.02152, train error 0.291429
	step 17000: train loss